# Note on this notebook
In this notebook I made a mistake in intrepreting the columns *price* and *retail_price*, as noted by the dataset creator. Please read the comments :)

# Predicting Successfull products
In this task, I will be identifying the characteristics that make a product sell. 

In the end, I built a really simple model (basically a linear regression) that indicates which products are more likely to return profit (based on the predicted sold_units and the difference between the retail_price and the price).

In this notebook I present the steps sequentially, only with a couple of functions. I should turn this into a proper pipeline in the future. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import re
from functools import reduce

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

sns.set()

In [ ]:
# The original data
df = pd.read_csv("../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv")

# EDA

In [ ]:
df.head().T

## Droping Duplicates
Removing duplicated entries

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
raw_df = df.copy() # Copy of the original Data after dropping the duplicates

## Checking missing values

In [ ]:
fig = px.bar(df.isna().sum().sort_values().reset_index(), x="index", y=0, title="Evaluating Missing values")
fig.update_layout({"yaxis": {"title": "Missing values"}})
fig.show()

# Checking columns Individually

## *merchant_profile_picture* and *merchant_has_profile_picture*
This column contains a link for the profile picture. No information can be easily obtained from here, unless we would do something with the pixels of the pictures (Not going to do that for now).
Since there is already a column indicating if the merchant has a profile picture, then I will just check that that column matches this one and if so, discard the *merchant_profile_picture* columns.

In [ ]:
df["merchant_profile_picture"].unique()

In [ ]:
print(df.loc[df["merchant_profile_picture"].isna(), "merchant_has_profile_picture"].unique())
print(df.loc[~df["merchant_profile_picture"].isna(), "merchant_has_profile_picture"].unique())

df.drop(["merchant_profile_picture"], axis=1, inplace=True)

## *has_urgency_banner* and *urgency_text*
It seems that there are only *urgency_text* in French, one indicating limited quantity and another indicating price reduction. Only one of them is "Réduction sur les achats en gros", as seen bellow. I wil check if this column matches the *has_urgency_banner*, and if so, discard this column. 

The column *has_urgency_banner* has values either 1 or nan, so I will turn the nan values of this column into 0.

In [ ]:
df["has_urgency_banner"].value_counts()

In [ ]:
df["urgency_text"].value_counts()

In [ ]:
print(df.loc[df["has_urgency_banner"].isna(), "urgency_text"].unique())
print(df.loc[~df["has_urgency_banner"].isna(), "urgency_text"].unique())

df.drop(["urgency_text"], axis=1, inplace=True)
df["has_urgency_banner"] = df["has_urgency_banner"].fillna(0)

## Checking product ratings

In this section we will check the columns:

- rating_one_count
- rating_two_count
- rating_third_count
- rating_fourth_count
- rating_five_count
- rating
- rating_count

The first 5 columns have missing values. Thee turned out to be products without any rating, altough for some reason the rating of these products is 5, event though they have a rating_count of 0. 

For these products, I will change the rating and all the rating counts to 0, indicating that they did not get any votes yet.

In [ ]:
rating_cols = ["rating_one_count", "rating_two_count", "rating_three_count", "rating_four_count", "rating_five_count", "rating", "rating_count"]

no_vote_df = df.loc[df[rating_cols].isna().any(axis=1), rating_cols]
df.loc[no_vote_df.index, rating_cols] = 0

df.loc[no_vote_df.index, rating_cols].head()

## *origin_country* and *countries_shipped_to*
The great majority of the dataset has origin in **CN**. This column has a really low variance, and so I will discard it, given that there is no statistical significant information that I can obtain from it.

*countries_shipped_to* is a numerical value. 


In [ ]:
df["origin_country"].value_counts()

In [ ]:
df["countries_shipped_to"].value_counts()

In [ ]:
df.drop("origin_country", axis=1, inplace=True)

## *product_color*, *product_picture* and *product_url*
There are 101 colors (see bellow), some of them appear very often while others do not. 

One option to deal with this would be to aggregate all the colors into similar groups (for example *denimblue* goes into *blue*), to reduce the number of categories.

However, I will replace the colors by the correspondent rgb vector (Hopefully, I will be able to the the rgb vectors for the majority of these colors from matplotlib.

The column *product_color* has missing values. I would fill them based on the *product_picture* columns, but none of those links work, which means that I will drop the column *product_picture* at the end anyway. 

The *product_url* link works, so I should be able to fill the missing colors from there. To do this, I will just go to the link and use a color picker on top of the product to get rgb (I could do this with a scrapper but this way it's faster by hand for now, specially because the product pages are hidden behind a login). 

Naturally, there will be products with different colors, in that case I will just select one of them based on the one that jumps to my attention first.

There are also 20 with color description multicolor. I will average out the color of these ones as well. 

There were two links that did not work, fot those I will assume the mean color of each rgb vector.


In [ ]:
df["product_color"].value_counts()

In [ ]:
# The urls of each product
df.loc[df["product_color"].isna(), ["product_url"]].head()

In [ ]:
missing_color_vectors = [
    "#F6F7F1",
    "#09041E",
    "#D24D41",
    "#6EACE1",
    "#323232",
    "#CECECE",
    "#E1E1EB",
    "#1D0E25",
    "#9EA59D",
    "#272727",
    "#FFD1DE",
    "#B37264",
    "#050505",
    "#120F1A",
    "#F9F9F9",
    "#C6363F",
    "#B7EDEC",
    "#4DA4C8",
    "#E7E6E4",
    "#C7955E",
    "#D3560A",
        np.nan,
    "#68166B",
    "#97E8D7",
    "#A6C2D1",
        np.nan,
    "#374757",
    "#E94875",
    "#EC2A13",
    "#4C4C4C",
    "#B9B4A9",
    "#0E0809",
    "#EBEEF4",
    "#DEEEEF",
    "#7F7181",
    "#E1E1E1",
    "#CFE5D8",
    "#E0D5D4",
    "#F4F7FB",
    "#74636D",
    "#E1E6EE"
]
assert len(missing_color_vectors) == df.loc[df["product_color"].isna(),:].shape[0], f"There are missing colors, {len(missing_color_vectors)}"
df.loc[df["product_color"].isna(), ["product_color"]] = missing_color_vectors
df.loc[df["product_color"] == "multicolor", ["product_color"]] = np.nan

In [ ]:
custom_color_rainbow = {
    "coffee": "#381E07",
    "floral": "#E4A593",
    "rose": "pink",
    "leopard": "#F2D24A",
    "leopardprint": "#F2D24A",
    "camouflage": "green",
    "army": "green",
    "camel": "#C19A6B",
    "wine": "#940F22",
    "apricot": "#F3C8AB",
    "burgundy": "#7D2F3D",
    "jasper": "#D0393C",
    "claret": "#940F22",
    "rainbow": "white",
    "star": "yellow",
    "nude": "pink"
}

def get_rgb(color):
    """ 
    Returns the rgb vector if the color exists on matplotlib. 
    This function is a bit messy with the nested try except statements, but performance is not critical 
    and it works for now, but I should clean it later """
    
    # TODO: THere is definitely a more elegant implementation
    if color in custom_color_rainbow.keys():
        return matplotlib.colors.to_rgb(custom_color_rainbow[color])

    try:
        return matplotlib.colors.to_rgb(color)
    except:
        base_colors = ["blue", "green", "red", "white", "black", "gold", "yellow", "pink", "purple", "orange", "grey", "khaki"]
        simplified_color = [c for c in base_colors if c in color]
        try:
            return matplotlib.colors.to_rgb(simplified_color[0])
        except:
            return np.nan
        
rgb_colors = df.loc[~df["product_color"].isna(), "product_color"].apply(get_rgb)

In [ ]:
rgb_colors_dict = [{"r": r[0], "g": r[1], "b": r[2]} for r in rgb_colors]
rgb_colors_dict_df = pd.DataFrame(rgb_colors_dict, index=rgb_colors.index)
df["r"] = rgb_colors_dict_df["r"]
df["g"] = rgb_colors_dict_df["g"]
df["b"] = rgb_colors_dict_df["b"]

# Fill the missing values with the mean of the column
df["r"].fillna(df["r"].mean(), inplace=True)
df["g"].fillna(df["g"].mean(), inplace=True)
df["b"].fillna(df["b"].mean(), inplace=True)

# Drop the useless columns
df.drop(["product_color", "product_url", "product_picture"], axis=1, inplace=True)

## *product_validation_size_id*
This is the size variation. I will first process the columns into XS, S, M, L and XL and afterwards I will use ordinal encoding on it. For the nan column, I will assume "M". 

There are of course products here on which this does not apply, given that these measures only apply to clothing. Still, if all the other despartments have the same size "M", that should not be a problem (At first glance anyway).

A lot of these sizes are from different scales, so I need to convert them to the same scale.

I used this side for this:
https://www.blitzresults.com/en/european-sizes/

In [ ]:
df["product_variation_size_id"].value_counts()

In [ ]:
def clean_sizes(s: str) -> str:
    return re.findall(r"M|X?[SsLl](?!\w+)", s)

def convert_us_to_eu(s: str) -> str:
    number = re.findall("\d+", s[0])[0]
    
    eu_to_letter = {
        (0, 36): "XS",
        (36, 40): "S",
        (40, 44): "M",
        (44, 48): "L",
        (48, 52): "XL",
        (52, 60): "XXL"
    }
    return [v for k, v in eu_to_letter.items() if k[0]<int(number)<k[1]][0]
     
original_sizes = df["product_variation_size_id"].dropna().unique()
changed_to_letter = [re.sub(r"EU\s*\d+", convert_us_to_eu, s) for s in original_sizes]
filtered_sizes = [clean_sizes(s) for s in changed_to_letter]

In [ ]:
original_sizes = df["product_variation_size_id"].dropna()
changed_to_letter = [re.sub(r"EU\s*\d+", convert_us_to_eu, s) for s in original_sizes]
filtered_sizes = [clean_sizes(s) for s in changed_to_letter]

In [ ]:
df.loc[original_sizes.index, "product_size"] = [c[0].lower() if c != [] else np.nan for c in filtered_sizes ]
df["product_size"].fillna("M", inplace=True)
df["product_size"].value_counts()

In [ ]:
df["product_size"] = OrdinalEncoder().fit_transform(df["product_size"].values.reshape(-1, 1))
df["product_size"].value_counts()

In [ ]:
df.drop("product_variation_size_id", axis=1, inplace=True)

## *merchant_id*, *merchant_info_subtitle*, *merchant_title* and *merchant_name*, *merchant_rating*, *merchant_rating_count*

- *merchant_id* is a a single id for each merchant and will be discarded
- *merchant_title* does not seem to contain much information and will be discarded
- *merchant_name* is just the name and will be discarded
- *merchant_info_subtitle* contains the same information as *merchant_rating_count* and another evaluation metric that does not seem to be the same as *merchant_rating*. Ultimately this column will be discarded


In [ ]:
df["merchant_id"].value_counts()

In [ ]:
df["merchant_title"].value_counts()

In [ ]:
# Surprisingly there are more unique values here than unique merchants
df["merchant_info_subtitle"].value_counts()

In [ ]:
df[["merchant_info_subtitle", "merchant_rating_count", "merchant_rating"]]

In [ ]:
df.describe()

In [ ]:
df.drop(["merchant_id", "merchant_title", "merchant_name", "merchant_info_subtitle"], axis=1, inplace=True)

## *currency_buyer*
All the currency is in Euro, so this column will be droped

In [ ]:
df["currency_buyer"].value_counts()

In [ ]:
df.drop("currency_buyer", inplace=True, axis=1)

## *tags*, *title*, *title_orig*
These are tags. A lot of interesting features can be drawn from here, together with the titles.
I will combine these three columns, and then try to create classes from the obtained words using some NLP tricks.

I will also create a column for the number of tags for each product.

In [ ]:
df["n_tags"] = df["tags"].apply(lambda x: len(x.split(",")))

### Processing the tags

In [ ]:
swords = stopwords.words('english')

def clean_text(s: str) -> str:
    """ Cleans the strings from the titles and the tags"""
    
    # Only Keep letters
    processed_s = re.sub(r"[^a-z]", " ", s.lower())
    
    ps = PorterStemmer()
    
    # stemmed words with Porter Lemantizer
    stemmed_s = [ps.stem(s) for s in processed_s.split()]
    
    unique_tags = list(set(stemmed_s))
    
    # Filter stop words
    cleaned_text = [w for w in unique_tags if (w not in swords and len(w) > 2)]
    
    return cleaned_text

all_tags = (df["tags"] + df["title"] + df["title_orig"]).values
processed_tags = [clean_text(s) for s in all_tags]

### Check all the unique tags
There are 3987 tags in this dataset

In [ ]:
len(set(reduce(lambda a,b : a+b, processed_tags)))

### Creating Columns for the relevant tags
We wil now use the created tags to create binary columns with the information that we want. Some of these columns can be:

- Men
- Women
- Shirt
- Robe
- Pajamas

etc..

(We need two columns for Man and Woman on the offchance that we find something for both and for neither genders)

In [ ]:
# Each of these will be a binary column
tags_list = [
    r"\bmen",
    r"\bwomen",
    "shirt",
    "robe",
    "dress",
    "skirt",
    "underwear",
    "swim",
    "nightwear",
    "sleepwear",
    "shorts"
]

def build_tags_dict(tags_list_per_product: list) -> dict:
    """ Returns a dict with 0 or 1, any of the tags_list were found  on the tags per sample"""
    return {tag.lstrip('\\b'): any(re.findall(tag, " ".join(tags_list_per_product))) for tag in tags_list}

#### Verifying the function the created routine

In [ ]:

testing_set = processed_tags[3]
print(build_tags_dict(testing_set))
print()
print(testing_set)

In [ ]:
cols_df = pd.DataFrame([build_tags_dict(t) for t in processed_tags])
cols_df.head()

In [ ]:
df = df.merge(cols_df, left_index=True, right_index=True)

In [ ]:
test_index = np.random.randint(df.shape[0])
df.loc[test_index, cols_df.columns.values.tolist() + ["title", "tags", "title_orig"]].to_dict()

#### Is the title equal to the original title
Check how frequently is the title the same as the original title

In [ ]:
(df["title"].str.lower() == df["title_orig"].str.lower() ).sum()

In [ ]:
df.drop(["tags", "title", "title_orig"], axis=1, inplace=True)

## *themes*, *crawl_month*  and *shipping_option_name*
Can be removed because the only contain one value or very few values

In [ ]:
df["theme"].value_counts()

In [ ]:
df["crawl_month"].value_counts()

In [ ]:
df.drop(["theme", "crawl_month", "shipping_option_name"], axis=1, inplace=True)

## *product_id*
Some product ids seem to be repeated, I will check if these products are actually repeated or not.

In [ ]:
df["product_id"].value_counts().value_counts()

The difference seems to be in the urgency banner column, everything else is the same. I will also check if they come from the same merchant, just to make sure, but then I will discard the products_ids duplicated that do not have th eurgency banner. 

In [ ]:
n_id_counts = df["product_id"].value_counts()
duplicated_ids = n_id_counts[n_id_counts > 1].index
comp_ids = df[df["product_id"].isin(duplicated_ids)].sort_values("product_id").set_index("product_id")
comp_ids.T

Verify the number of duplicated product ids per merchant. If there was any 1 here, it meant that some of the same product ids were allocated to different merchants


In [ ]:
raw_df.loc[raw_df["product_id"].isin(duplicated_ids), ["merchant_id", "product_id"]].groupby("merchant_id").count().squeeze().min()

Drop the duplicated that do not have the urgency banner

In [ ]:
df = df.sort_values("has_urgency_banner", ascending=False).drop_duplicates("product_id")
df.drop("product_id", inplace=True, axis=1)

## Convert binary columns to boolean
As a final step, we will convert all the binary columns to boolean

In [ ]:
df.describe().T

In [ ]:

to_bool_cols = ["uses_ad_boosts", "shipping_is_express", "badge_local_product", "badge_product_quality", "has_urgency_banner", "merchant_has_profile_picture"]
df[to_bool_cols] = df[to_bool_cols].astype(bool)

In [ ]:
df.dtypes

## Assert that there are no missing values

In [ ]:
assert not df.isna().any().any()

## Check if there are products out of stock
It is important to figure out if there are no products out of stock, because if this is the case then that could mean that some products did not sell because they were out of stock. That does not seem to be the case fortunately. 

In [ ]:
df["inventory_total"].min()

# Modelling
We now have only numeric columns, now we can and take some conclusions from the data

In [ ]:
# Just some usefull variables
cont_cols = df.select_dtypes(exclude="bool").columns
bool_cols = df.select_dtypes("bool").columns

## Correlation
There seems to be a really strong positive correlation  between the rating_counts and the number of units sold. 

This basically means that products that generate more traffic and have been around for longer have sold more units.

Given that there is a lot of correlation between the individual score counts, and the total rating couts, I will remove the five individual score counts.

Also, it seems that the *units_sold* is not exactly a continuous columns, but rather a categorical column. Still, I will treat this problem as a regression problem. 

In [ ]:
px.imshow(df[cont_cols].corr(), width=1000, height=1000)

In [ ]:
df.drop(["rating_five_count", "rating_four_count", "rating_three_count", "rating_two_count", "rating_one_count"], axis=1, inplace=True)

## Scatter Matrix

In [ ]:
scatter_matrix_cols = ["price", "units_sold", "rating", "merchant_rating", "rating_count"]
px.scatter_matrix(df[scatter_matrix_cols], width=1000, height=1000)

## Checking the boolean columns
Now we can check the boolean columns to see which sell the most

In [ ]:
px.box(df[bool_cols.values.tolist() + ["units_sold"]].melt(id_vars="units_sold"), x="variable", y="units_sold", color="value", title="Sold Unites Based on the Boolean Columns")

On average, it seems that when you have profile picture, and "badge product quality" and the swim and nightwear tags you will sell more. However, it is important to note that these distinctions are highly unbalanced.

## Regression analysis
We will now fit a linear regression with regularization (elasticnet) to try and figure out, which features are actually more important.

In [ ]:
X = df.drop("units_sold", axis=1)
y = df["units_sold"]
reg = make_pipeline(StandardScaler(), ElasticNet(alpha=0.5))
reg.fit(X, y)
pd.Series(reg[-1].coef_, index=X.columns).sort_values().plot.bar(figsize=(20, 5))

r2 = r2_score(y, reg.predict(X))
plt.title(f"R2: {round(r2, 2)}")
plt.show()

In [ ]:
no_counts_df = df.drop(["rating_count", "merchant_rating_count"], axis=1)

### Regression analysis conclusion
The rating_count is overwhelmingly important for the units_sold, which to me only indicates that products that have been around for longer (Hence have more ratings) have sold more. For the reason, I decided to remove this column from the remainder of the anaylsis (as well as the *merchant_rating_count*).

The results changed without this column, and we were only able to get an R2 of 7%. 

In [ ]:
X = no_counts_df.drop("units_sold", axis=1)
y = no_counts_df["units_sold"]
reg = make_pipeline(StandardScaler(), ElasticNet(alpha=0.5))
reg.fit(X, y)
pd.Series(reg[-1].coef_, index=X.columns).sort_values().plot.bar(figsize=(20, 5))

r2 = r2_score(y, reg.predict(X))
plt.title(f"R2: {round(r2, 2)}")
plt.show()

##  Conclusion Regression analysis
Using the regression analysis we concluded that the number of ratings is the most important factor for determining the number of units sold. 

I believe this is however, not that relevant, because it merely implies that products that have been on store for longer, sold more. 

When I removed this column, the R2 square score lowered significantly.

Also, none of the tags I created seem to have that much of an effect. 


# Model for selecting products
Now I will create a model that, identifies which products should be bought. This model will be a regression model using the available information, to predict the number of units sold. Then, the number of units sold will be multiplied by the difference between price and retail price. 

To indicate if more of some product should be aquired, then the predicted sold amount will be subtracted from the number of units currently in stock.

There seem to be negative differences between the retail and price column, indicating that some products are aquired at a more expensive that they are sold?? Still, I will not change the values of this column

In [ ]:
df["profit"] = df["retail_price"] - df["price"]
df["profit"].hist(bins=40, figsize=(20, 5))
plt.title("Difference between retail_price and price")
plt.show()

## Create the regression model using only the rating_counts column
Ordinarly I would use cross validation for model testing, but in here I am going to go with normal train test split because it will be easier to integrate this to get the results

In [ ]:
X = df["rating_count"].to_frame()
y = df["units_sold"]


X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
reg = make_pipeline(StandardScaler(), ElasticNet(0.5))
reg.fit(X_train, y_train)
predition = reg.predict(X_test)
test_r2 = r2_score(y_test, predition)
print(f"Test r2_score: {round(test_r2, 2)}")

In [ ]:
# Make sold units prediction
def get_results(reg, df: pd.DataFrame) -> pd.DataFrame:
    """
    reg is the pre trained pipelin (In this case with the included scaler)
    df is the original Data Frame
    """
    df["predicted_sold_units"] = reg.predict(df["rating_count"].to_frame())

    df["profit"] = (df["retail_price"] - df["price"]) * df["predicted_sold_units"]

    df["units_to_order"] = np.ceil(np.min(df["predicted_sold_units"] - df["inventory_total"], 0))
    return df

results_df = get_results(reg, raw_df)

# Most profitable products

In [ ]:
profitable_products = results_df[["profit", "product_id"]].set_index("product_id").squeeze().sort_values(ascending=False)
profitable_products.head(10)

In [ ]:
results_df["profit"].hist(bins=100, figsize=(20, 5))

# Products to order
Get the units to order from the most profitable products

In [ ]:
results_df.loc[results_df["product_id"].isin(profitable_products.head().index), ["units_to_order", "product_id"]]

# Conclusion
In this notebook I build a simple model (a simple linear regression) to indicate which products are more profitable and how many units of these products should be ordered. 

I find this problem really interesting, I wouls really like to work more on it with more data, to build a system like a stock optimization algorithm and something similar. 

I did not turn this into a proper pipeline, because I feel like this is it is easier to understand and change the notebook, but maybe in the future I will do so. 